In [5]:
import os,json
if not os.path.isdir("workspace"):
    os.mkdir("workspace")
    
OBJDIR = os.path.join(os.environ["USE_KH2_GITPATH"], "KH2", "obj")
lines = open("enemy_records.csv").read().split("\n")
headers = lines[0].split(",")
records = {}

for line in lines[1:]:
    if len(line) == 0:
        continue
    record = {}
    for i in range(len(headers)):
        record[headers[i]] = line.split(",")[i]
    record["ucm"] = record["ucm"].zfill(4)
    record["spawns"] = [i.zfill(4) for i in record["spawns"].split(';') if len(i) > 0]
    name = record["name"].strip()
    record["name"] = name
    records[name] = record
    
def getsize(name):
    size = 0
    record = records[name]
    size = os.path.getsize(os.path.join(OBJDIR,record["model"]+".mdlx"))
    fmfn = os.path.join(OBJDIR,record["model"]+".fm")
    if os.path.exists(fmfn):
        size += os.path.getsize(fmfn)
    anfn = record["animation"]
    if len(anfn) == 0:
        anfn = record["model"] + ".mset"
    anfn = os.path.join(OBJDIR, anfn)
    if os.path.exists(anfn):
        size += os.path.getsize(anfn)
    for spawn in record["spawns"]:
        for r in records:
            if records[r]["ucm"] == spawn:
                size += getsize(records[r]["name"])
    return size

for r in records:
    records[r]["size"] = getsize(r)
records
json.dump(records, open(os.path.join("workspace", "enemy_records_filled.json"), "w"), sort_keys=True, indent=4, separators=(',', ': '))

In [6]:
for r in sorted(records.values(), key=lambda k: k["size"]):
    print("{} {}".format(r["size"], r["name"]))

552 Invisible Target
6980 Nothing
10952 Lingering Will's Ultima Cannon Shot
41084 Bees
73460 Rapid Thruster AL
77342 Tiny Pyramid Dummy
78392 Gargoyle Knight
79594 Gargoyle Warrior
84720 Card Sora
87118 Nobody from Final Battle v1
97616 Dice Sora
102818 Zexions Book
104980 Oogie Box Dummy
109420 Emerald Blues NM
113602 Roxas Oblivion Keyblade
114002 Roxas Oathkeeper Keyblade
122640 Fiery Globe
134096 Jafar Flying Wreckage
180522 Fiery Globe Volcano Lord
186364 Spring Metal
189784 Icy Cube
199256 Berserker Sword
207636 Fire Attack
212176 Silver Rock
237820 Saix's Claymore
238004 Data Saix Claymore
245100 Aeroplane WI
251012 Rapid Thruster
258308 Illuminator
272536 Barrel Dummy
279204 Lock Dummy
280926 Cannon Gun
285258 Crimson Jazz AL
290842 Crimson Jazz
292560 Hook Bat AL
296752 Hook Bat
298036 Beffudler
298980 Shock Dummy
337392 Bolt Tower
343104 Crescendo
352940 Runemaster
363704 Strafer
369292 Surveillance Robot
374008 Zexions Something
382620 Magnum Loader
398180 Luxord's Card
4007

In [ ]:
def lookupEnemySize(enemy):
    if len(enemy) == 4:
        aiparam = ''
    if len(enemy) == 6:
        aiparam = enemy[1]
        enemy = enemy[2:]
    for e in records.values():
        if e["ucm"].lower().zfill(4) == enemy.lower():
            if e["aiparam"] == '0' or e["aiparam"] == '':
                if aiparam == '0' or aiparam == '':
                    return e["size"]
            if e["aiparam"] == aiparam:
                return e["size"]
    raise Exception("enemy not found {}".format(enemy))
lookupEnemySize('035E')